In [ ]:
!pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 12.0 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import tensorflow as tf
import cv2

from IPython.display import clear_output,display
from IPython.display import Image
import matplotlib.pyplot as plt

from keras import Model, Input
from keras.models import clone_model
from keras.layers import Dense

In [ ]:
env=gym.make("CartPole-v1",render_mode="rgb_array")

In [ ]:
#policy
#here state is going to be a vector of 4 elements
def policy(state,epsilon=0.0):
  #here we'll batch it up i.e. convert it into a batch of oonly one state
  # tenor of 1 to tensor of [1,4]
  action = tf.argmax(q_net(tf.expand_dims(state,axis=0))[0],output_type=tf.int32)
  if(tf.random.uniform(shape=(),maxval=1)<=epsilon):
    action = tf.random.uniform(shape=(),maxval=2,dtype=tf.int32)
  return action

In [ ]:
def calculate_reward(state):
    # -1 Reward for every step
    reward = -1.0

    # Reward +1.0 when,
    # Cart Position is between +-0.5,
    # Cart Velocity is within +-1,
    # Pole Angle is lesser than 4 degrees, and
    # pole angular velocity is lower than 15%
    if -0.5 <= state[0] <= 0.5 and -1 <= state[1] <= 1 and -0.07 <= state[2] <= 0.07 and -0.525 <= state[3] <= 0.525:
        reward = 1.0

    return reward

In [ ]:
def insert_transition(transition):
  if(len(replay_buffer)>max_transition):
    replay_buffer.pop(0)
  replay_buffer.append(transition)

In [ ]:
def sample_transition(batch_size=16):
  random_indices=tf.random.uniform(shape=(batch_size,),minval=0,maxval=len(replay_buffer),dtype=tf.int32)
  sampled_current_states=[]
  sampled_actions=[]
  sampled_rewards=[]
  sampled_next_states=[]
  sampled_terminals=[]

  for index in random_indices:
    sampled_current_states.append(replay_buffer[index][0])
    sampled_actions.append(replay_buffer[index][1])
    sampled_rewards.append(replay_buffer[index][2])
    sampled_next_states.append(replay_buffer[index][3])
    sampled_terminals.append(replay_buffer[index][4])
  return tf.convert_to_tensor(sampled_current_states), tf.convert_to_tensor(sampled_actions), tf.convert_to_tensor(sampled_rewards), tf.convert_to_tensor(sampled_next_states), tf.convert_to_tensor(sampled_terminals)


In [ ]:
net_input =Input(shape=(4,))
x=Dense(62,activation="relu")(net_input)
x=Dense(62,activation="relu")(x)
output=Dense(2,activation="linear")(x)
q_net=Model(inputs=net_input,outputs=output)

#learning rate iis managed by optimizer instead of alpha
q_net.compile(optimizer="adam")
loss_fn=tf.keras.losses.Huber()
target_net = clone_model(q_net)

def get_q_values(states):
    return tf.reduce_max(q_net(states), axis=1)


epsilon=1.0
epsilon_decay=1.005
gamma=0.99
episodes=400
batch_size=64
max_transition=10000
replay_buffer=[]
target_update_after=4
LEARN_AFTER_STEPS=3
# Gathering Random initial states for Average Q Metric
random_states = []
done = False
i = 0
state,_ = env.reset()
while i < 20 and not done:
    random_states.append(state)
    state, _, done, _ = env.step(policy(state).numpy())
    i += 1

random_states = tf.convert_to_tensor(random_states)


step_counter=0
metric = {"episode": [], "length": [], "total_reward": [], "avg_q": [], "exploration": []}
for episode in range (episodes):
  done=False
  state,_=env.reset()
  total_rewards=0
  episode_length=0
  while not done:
    # print(f"State: {state}, Type: {type(state)}")
    action=policy(state,epsilon)
    next_state,_,terminated,truncated,info=env.step(action.numpy())
    reward= calculate_reward(next_state)
    done=terminated or truncated
    insert_transition([state,action,reward,next_state,done])
    state=next_state
    step_counter+=1

    if step_counter % LEARN_AFTER_STEPS == 0:
      # train the Q-network
      current_states,actions,rewards,next_states,terminals=sample_transition(batch_size)

      next_action_values= tf.reduce_max(target_net(next_states),axis=1)
      targets =tf.where(terminals,rewards, rewards+gamma*next_action_values)
      with tf.GradientTape() as tape:
        preds = q_net(current_states)
        batch_nums = tf.range(0, limit=batch_size)
        indices =  tf.stack((batch_nums, actions), axis=1)
        current_value = tf.gather_nd(preds, indices)
        loss=loss_fn(targets,current_value)
      grads=tape.gradient(loss,q_net.trainable_weights)
      q_net.optimizer.apply_gradients(zip(grads,q_net.trainable_weights))

    if(step_counter%target_update_after==0):
      target_net.set_weights(q_net.get_weights())
    total_rewards+=reward
    episode_length+=1

  print("Epiosde: ",episode," Length: ",episode_length," Reward: ",total_rewards, "Epsilon: ",epsilon)
  # Saving Metrics
  avg_q = tf.reduce_mean(get_q_values(random_states)).numpy()
  metric["episode"].append(episode)
  metric["length"].append(episode_length)
  metric["total_reward"].append(total_rewards)
  metric["avg_q"].append(avg_q)
  metric["exploration"].append(epsilon)
  epsilon/=epsilon_decay
  pd.Dataframe(metric).to_csv("metrics.csv",index=False)
env.close()
q_net.save("dqn_q_net")

Epiosde:  0  Length:  18  Reward:  18.0 Epsilon:  1.0
Epiosde:  1  Length:  48  Reward:  48.0 Epsilon:  0.9950248756218907
Epiosde:  2  Length:  41  Reward:  41.0 Epsilon:  0.990074503106359
Epiosde:  3  Length:  26  Reward:  26.0 Epsilon:  0.98514875930981
Epiosde:  4  Length:  78  Reward:  78.0 Epsilon:  0.9802475217013037
Epiosde:  5  Length:  19  Reward:  19.0 Epsilon:  0.9753706683595063
Epiosde:  6  Length:  10  Reward:  10.0 Epsilon:  0.970518077969658
Epiosde:  7  Length:  21  Reward:  21.0 Epsilon:  0.9656896298205554
Epiosde:  8  Length:  17  Reward:  17.0 Epsilon:  0.9608852038015477
Epiosde:  9  Length:  19  Reward:  19.0 Epsilon:  0.9561046803995501
Epiosde:  10  Length:  24  Reward:  24.0 Epsilon:  0.9513479406960699
Epiosde:  11  Length:  34  Reward:  34.0 Epsilon:  0.9466148663642487
Epiosde:  12  Length:  27  Reward:  27.0 Epsilon:  0.9419053396659193
Epiosde:  13  Length:  16  Reward:  16.0 Epsilon:  0.937219243448676
Epiosde:  14  Length:  13  Reward:  13.0 Epsilon: 

### **Why Do We Need This Training Step?**
The goal of reinforcement learning is to make the Q-network better at predicting the "value" of taking certain actions in certain states.

- We want the Q-network's predicted Q-values (`preds`) to match the "target" Q-values (`targets`), which represent what the true value should ideally be.
- This training process is like telling the Q-network: *"Hey, you guessed this, but the correct answer should be that. Update yourself to be more accurate next time!"*

---

### **What Are We Doing Here?**

1. **Predict the Value of the Next State (Future Rewards):**
   ```python
   next_action_values = tf.reduce_max(target_net(next_states), axis=1)
   ```
   - Using the **target network**, we look at the next states (`next_states`) from our sampled transitions and predict the Q-values for all possible actions in those states.
   - `tf.reduce_max` takes the maximum Q-value for each next state. This is because we assume the agent will act optimally in the future (choose the best action).

2. **Calculate Target Q-Values for the Current Step:**
   ```python
   targets = tf.where(terminals, rewards, rewards + gamma * next_action_values)
   ```
   - If the sampled transition ends in a terminal state (`terminals` is `True`), the target Q-value is simply the `reward` since there are no future rewards to consider.
   - If the transition is not terminal, the target Q-value is:
     \[
     \text{Target} = \text{Reward} + \gamma \times \text{(Max Q-value of the next state)}
     \]
     - `gamma` is the discount factor, which ensures that rewards further in the future are less important than immediate rewards.

3. **Predict the Q-Values for Current States:**
   ```python
   preds = q_net(current_states)
   ```
   - The **main network** (`q_net`) predicts Q-values for all actions in the `current_states`.

4. **Extract the Q-Values for the Actions Taken:**
   ```python
   batch_nums = tf.range(0, limit=batch_size)
   indices = tf.stack((batch_nums, actions), axis=1)
   current_value = tf.gather_nd(preds, indices)
   ```
   - The network predicted Q-values for **all possible actions**, but we only care about the Q-values for the actions that were actually taken in our sampled transitions.
   - Here’s how we extract those specific Q-values:
     - `batch_nums` generates indices for the batch of transitions (e.g., `[0, 1, 2, ...]`).
     - `actions` are the actions that were taken in the transitions.
     - `tf.stack((batch_nums, actions), axis=1)` creates indices to identify the specific Q-values corresponding to the actions in each transition.
     - `tf.gather_nd(preds, indices)` retrieves the Q-values for those actions.

---

### **Why Are We Doing This?**

We now have two things:
1. **Targets**: What the Q-values *should* be (based on the Bellman equation and the target network).
2. **Predictions**: What the Q-network *currently* thinks the Q-values are.

The next step (not shown in your code snippet) would typically involve calculating the **loss** between these two values and using it to update the Q-network. For example:
```python
loss = tf.reduce_mean((targets - current_value) ** 2)
optimizer.minimize(loss, q_net.trainable_variables)
```

---

### **In Layman Terms**

After collecting experience from the environment, we:
1. Use the **target network** to estimate the "correct answers" (targets) for the Q-values.
2. Use the **main network** to get the current guesses (predictions) for the Q-values.
3. Compare the two and update the main network so its guesses become more accurate.

Evaluator

In [ ]:
for episode in range(5):
  done = False
  observation,_=env.reset()
  state= tf.convert_to_tensor([observation])
  while not done:
    frame=env.render()
    clear_output(wait=True)
    plt.imshow(frame)
    plt.axis('off')
    display(plt.gcf())
    action=policy(state);
    state,reward,terminated,truncated,info=env.step(action.numpy())
    done=terminated or truncated
    state=tf.convert_to_tensor([state])
env.close()